# 구문 분석: Part IV - Chart Parser

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 참고 사이트 
    - nltk book 8.Analyzing sentence structure(https://www.nltk.org/book/ch08.html) 내용을 참고해서 자료를 만듦. 
    - nltk book의 8장 예제를 일부 사용.
- 참고 서적
    - Natural Language Processing with Python Cookbook, Krishna Bhavsar, Naresh Kumar, Pratap Dangeti, Packt Publishing, 2017.
    - Chapter 6, Chapter 7의 예제를 일부 사용.

### Prerequisite
동영상 강의(6장-하향식 구문분석 및 상향식 구문분석)을 시청한 후에 실습을 진행해야 함.

여기서 다룰 내용은
- 다양한 문장 구조를 표현하려면 형식 문법을 어떻게 사용하면 될까? 
- 구문 트리를 사용하여 문장 구조를 어떻게 보여줄 수 있을까?
- 파서가 문장을 어떻게 분석하고, 구문 트리를 자동으로 만들어 낼까?

In [ ]:
import nltk

## 3.4   부분문자열 테이블(Well-Formed Substring Tables, wfst) - Chart Parsing
- 위에서 살펴 본 간단한 파서는 완전함(completeness)과 효율 측면에서 한계가 있다. 이를 해결하기 위해 동적 프로그래밍 기법을 구문분석 문제에 적용할 것이다. 
    - 동적 프로그래밍은 중간 연산 결과를 저장하고 필요할 때 다시 사용함으로써 효율을 향상시킬 수 있다. 이를 구문분석에 적용 할 수 있다. 
    - 즉, 구문분석 작업에 대한 중간 처리 결과를 저장하고, 필요에 따라 이 내용을 참조하여 완전한 결과를 얻을 수 있다. 
    - 이러한 구문분석 접근 방식을 chart parsing이라고 한다. 
- 동적 프로그래밍에서는 in my pajamas와 같은 전치사구(PP)는 한 번만 만들면 된다. 
    - 처음 이 구문을 분석하면서 이 결과를 테이블에 저장하고, NP나 VP의 하위 구성요소로 사용해야 할 때 테이블에서 이를 찾으면 된다. 
    - 이 테이블을 WFST(well-formed substring table)로 부른다. 여기서 "substring"은 문장 내에서 연속적인 단어 시퀀스를 말한다. 
    - 발견한 구문 구성요소를 체계적으로 WFST에 상향식(bottom-up)으로 기록하는 과정을 살펴볼 것이다.

In [ ]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

In [ ]:
text = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
p = groucho_grammar.productions(rhs=text[6]) #pajamas를 rhs로 갖는 생성규칙 : N->'pajamas'

print(len(p))
print(p[0]) #생성규칙이 여러 개이면, 그 중 첫 번째 생성규칙을 선택.
print(p[0].lhs())
print(p[0].rhs())

### wfst  초기화
- 2차원 테이블 = (numtoken+1) x (numtoken+1)
- 테이블의 모든 값은 None으로 초기화.
- 각 행은 문장에 나타난 token 순서와 같음. 생성규칙에서 token을 rhs로 갖는 생성규칙의 lhs를 할당
    - 대각선에 해당하는 부분만 lhs가 할당됨.
- wfst의 첫 번째 열과 마지막 행(numtoken+1)은 모두 None을 갖고 있음.

In [ ]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        wfst[i][i+1] = productions[0].lhs()
    return wfst

In [ ]:
import numpy as np

tokens = "I shot an elephant in my pajamas".split()
wfst0 = init_wfst(tokens, groucho_grammar)

print(tokens)
print(len(tokens), np.ndim(wfst0))

wfst 테이블을 출력해보자.

In [ ]:
from IPython.display import display

display(wfst0, tokens)

display 함수를 재정의하여 보기좋게 출력되도록 만들어 보자.

In [ ]:
def display(wfst, tokens):
    print('\nWFST ' + ' '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

In [ ]:
display(wfst0, tokens)

In [ ]:
print(len(groucho_grammar.productions()))
index = dict((p.rhs(), p.lhs()) for p in groucho_grammar.productions())
print(index)

- 상향식 구문분석으로 진행된다. 
    - 생성규칙 오른쪽에 있는 패턴을 찾아, 이 생성규칙 왼쪽에 있는 Nonterminal로 변환 가능한 지를 grammar에서 확인한다. 

### wfst 구성 과정  : for span in range(2, 7+1) 에서 span=2일 때
- span=2, start=0, end=2, mid=1 대각선으로 한 칸 떨어진 2개의 Nonterminal(NP, V) 비교
    - (NP, V) = wfst0[0][1], wfst0[1][2]. (NP, V)는 생성규칙의 rhs에 없기 때문에 lhs를 찾을 수 없음. 
- span=2, start=1, end=3, mid=2 대각선으로 한 칸 떨어진 2개의 Nonterminal (V, Det) 비교
    - (V, Det) = wfst0[1][2], wfst0[2][3]. (V, Det)는 생성규칙의 rhs에 없기 때문에 lhs를 찾을 수 없음.
- span=2, start=2, end=4, mid=3 대각선으로 한 칸 떨어진 2개의 Nonterminal (Det, N) 비교
    - (Det, N) = wfst0[2][3], wfst0[3][4]. (Det, N)은 생성규칙의 rhs에 있으며, **lhs인 NP를 찾을 수 있음.**
- span=2, start=3, end=5, mid=4 대각선으로 한 칸 떨어진 2개의 Nonterminal (N, P) 비교
    - (N, P) = wfst0[3][4], wfst0[4][5]. (N, P)는 생성규칙의 rhs에 없으며, lhs를 찾을 수 없음.
- span=2, start=4, end=7, mid=5 대각선으로 한 칸 떨어진 2개의 Nonterminal (P, Det) 비교
    - (P, Det) = wfst0[4][5], wfst0[5][6]. (P, Det)는 생성규칙의 rhs에 없으며, lhs를 찾을 수 없음.
- span=2, start=5, end=8, mid=6 대각선으로 한 칸 떨어진 2개의 Nonterminal (Det, N) 비교
    - (Det, N) = wfst0[5][6], wfst0[6][7]. (Det, N)은 생성규칙의 rhs에 있으며, **lhs인 NP를 찾을 수 있음.**

In [ ]:
span = 2
for start in range(7+1-span):
    end = start + span
    for mid in range(start+1, end):
        print(span, start, mid, end, end=' ')
        nt1, nt2 = wfst0[start][mid], wfst0[mid][end]  
        print(nt1, nt2, end=' ')
            
        if nt1 and nt2 and (nt1,nt2) in index:
            wfst0[start][end] = index[(nt1,nt2)]
            print('lhs=', wfst0[start][end], end=' ')            
    print()

2군데에서 생성규칙 오른쪽에 있는 패턴(**Det N**)을 찾아 이 생성규칙의 왼쪽에 있는 Nonterminal **NP**를 테이블에 추가했다.

In [ ]:
display(wfst0, tokens)

### wfst 구성 과정  : for span in range(2, 7+1) 에서 span=3일 때
- 이번에는 2칸씩 이동하면서 해당 패턴이 생성규칙의 rhs에 있는지를 확인한다.
- 이번에는 
    - **V NP** 를 **VP**로, 
    - **P NP** 를 **PP**로 변환이 가능함을 알 수 있다.

In [ ]:
span = 3
for start in range(7+1-span):
    end = start + span
    for mid in range(start+1, end):
        print(span, start, mid, end, end=' ')
        nt1, nt2 = wfst0[start][mid], wfst0[mid][end]  
        print(nt1, nt2, end=' ')
            
        if nt1 and nt2 and (nt1,nt2) in index:
            wfst0[start][end] = index[(nt1,nt2)]
            print('lhs=', wfst0[start][end])  
    print()

In [ ]:
display(wfst0, tokens)

이제 제대로된 함수 complete_wfst를 완성해 보자.

In [ ]:
def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst

In [ ]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar)
display(wfst1, tokens)

**trace=True** : 생성규칙의 rhs 패턴을 찾아 해당 생성규칙의 Nonterminal을 추가한 사례를 확인할 수 있다.

In [ ]:
wfst1 = complete_wfst(wfst0, tokens, groucho_grammar, trace=True)

이번에는 ChartParser 라이브러리를 사용하여 구현해보자.

In [ ]:
grammar2 = nltk.CFG.fromstring("""
S -> NP VP
NP -> NNP VBZ
VP -> IN NNP | DT NN IN NNP
NNP -> 'Incheon' | 'Songdo' | 'Seoul' | 'Korea'
VBZ -> 'is'
IN -> 'in' | 'of'
DT -> 'the'
NN -> 'capital'
""")

In [ ]:
from nltk.parse.chart import ChartParser, BU_LC_STRATEGY

cp = ChartParser(grammar2, BU_LC_STRATEGY, trace=True)

tokens = "Seoul is the capital of Korea".split()

In [ ]:
chart = cp.chart_parse(tokens)
parses = list(chart.parses(grammar2.start()))
print("Total Edges :", len(chart.edges()))

In [ ]:
for tree in parses: 
    print(tree)
    tree.draw()